In [6]:
import pandas as pd
from io import StringIO

dados = ['201.csv','202.csv','211.csv','212.csv','221.csv','222.csv','231.csv','232.csv','241.csv','242.csv']
lista_vazia = []

for arquivo in dados:
    try:
        print(f"Lendo arquivo: {arquivo}")
        with open(arquivo, 'r', encoding='utf-8') as f:
            linhas = f.readlines()

        num_campos_esperados = linhas[0].count(';') + 1
        linhas_validas = [linha for linha in linhas if linha.count(';') + 1 == num_campos_esperados]
        print(f"Linhas totais: {len(linhas)} - Linhas válidas: {len(linhas_validas)}")

        if len(linhas_validas) == 0:
            print(f"Nenhuma linha válida encontrada em {arquivo}, pulando arquivo.")
            continue

        csv_filtrado = StringIO(''.join(linhas_validas))
        df_temp = pd.read_csv(csv_filtrado, sep=';', low_memory=False)
        lista_vazia.append(df_temp)

    except Exception as e:
        print(f"Erro ao processar {arquivo}: {e}")
        pass

if len(lista_vazia) == 0:
    print("Nenhum DataFrame foi carregado. Verifique os arquivos e filtros.")
else:
    df = pd.concat(lista_vazia, ignore_index=True)
    print(df.head())

# Depois, se quiser concatenar todos num DataFrame só:
df = pd.concat(lista_vazia, ignore_index=True)
print(df.head())

Lendo arquivo: 201.csv
Erro ao processar 201.csv: [Errno 2] No such file or directory: '201.csv'
Lendo arquivo: 202.csv
Erro ao processar 202.csv: [Errno 2] No such file or directory: '202.csv'
Lendo arquivo: 211.csv
Erro ao processar 211.csv: [Errno 2] No such file or directory: '211.csv'
Lendo arquivo: 212.csv
Erro ao processar 212.csv: [Errno 2] No such file or directory: '212.csv'
Lendo arquivo: 221.csv
Erro ao processar 221.csv: [Errno 2] No such file or directory: '221.csv'
Lendo arquivo: 222.csv
Erro ao processar 222.csv: [Errno 2] No such file or directory: '222.csv'
Lendo arquivo: 231.csv
Erro ao processar 231.csv: [Errno 2] No such file or directory: '231.csv'
Lendo arquivo: 232.csv
Erro ao processar 232.csv: [Errno 2] No such file or directory: '232.csv'
Lendo arquivo: 241.csv
Erro ao processar 241.csv: [Errno 2] No such file or directory: '241.csv'
Lendo arquivo: 242.csv
Erro ao processar 242.csv: [Errno 2] No such file or directory: '242.csv'
Nenhum DataFrame foi carregado

ValueError: No objects to concatenate

In [4]:
df['Data Coleta'] = pd.to_datetime(df['Data da Coleta'], format='%d/%m/%Y')
df['Ano-Mês'] = df['Data Coleta'].dt.to_period('M')

NameError: name 'df' is not defined

In [3]:

# Converter coluna de data para datetime e criar coluna Ano-Mês
df['Data Coleta'] = pd.to_datetime(df['Data da Coleta'], format='%d/%m/%Y')
df['Ano-Mês'] = df['Data Coleta'].dt.to_period('M')

# 1. Evolução mensal do preço médio da gasolina por estado
evolucao_gasolina = df.loc[df['Produto'] == 'GASOLINA COMUM', :].groupby(['Estado - Sigla', 'Ano-Mês'])['Valor de Venda'].mean().reset_index()

# 2. Comparação entre gasolina comum e aditivada
comparativo = df.loc[df['Produto'].isin(['GASOLINA COMUM', 'GASOLINA ADITIVADA']), :].groupby(['Produto', 'Ano-Mês'])['Valor de Venda'].mean().reset_index()

# 3. Análise de variação de preços por região
regioes = {
    'N': ['AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO'],
    'NE': ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],
    'CO': ['DF', 'GO', 'MT', 'MS'],
    'SE': ['ES', 'MG', 'RJ', 'SP'],
    'S': ['PR', 'RS', 'SC']
}

def estado_para_regiao(sigla):
    for regiao, estados in regioes.items():
        if sigla in estados:
            return regiao
    return 'Outro'

df['Região'] = df['Estado - Sigla'].apply(estado_para_regiao)

variacao_regiao = df.loc[df['Produto'] == 'GASOLINA COMUM', :].groupby(['Região', 'Ano-Mês'])['Valor de Venda'].mean().reset_index()

# 4. Postos com maiores e menores preços (gasolina comum)
postos_extremos = df.loc[df['Produto'] == 'GASOLINA COMUM', ['Revenda', 'Municipio', 'Estado - Sigla', 'Valor de Venda']]

maiores_precos = postos_extremos.sort_values(by='Valor de Venda', ascending=False).head(10)
menores_precos = postos_extremos.sort_values(by='Valor de Venda').head(10)

# Perguntas analíticas

# Estado com maior variação entre 2020 e 2024
df_gas = df.loc[df['Produto'] == 'GASOLINA COMUM', :].copy()
df_gas['Ano'] = df_gas['Data Coleta'].dt.year

inicio_fim = df_gas.loc[df_gas['Ano'].isin([2020, 2024]), :].groupby(['Estado - Sigla', 'Ano'])['Valor de Venda'].mean().unstack()
inicio_fim['Variação (%)'] = ((inicio_fim[2024] - inicio_fim[2020]) / inicio_fim[2020]) * 100
maior_var = inicio_fim.sort_values(by='Variação (%)', ascending=False).head(1)

# Estados onde o etanol é competitivo (< 70% do preço da gasolina)
media_precos = df.loc[df['Produto'].isin(['GASOLINA COMUM', 'ETANOL']), :].groupby(['Estado - Sigla', 'Produto'])['Valor de Venda'].mean().unstack()
media_precos['Proporção'] = media_precos['ETANOL'] / media_precos['GASOLINA COMUM']
estados_competitivos = media_precos.loc[media_precos['Proporção'] < 0.7, :]

# Diferença significativa entre preços das distribuidoras (gasolina comum)
df_dist = df.loc[df['Produto'] == 'GASOLINA COMUM', :]
media_dist = df_dist.groupby('Distribuidora')['Valor de Venda'].mean().reset_index().sort_values(by='Valor de Venda', ascending=False)

# Resultados básicos
print("Evolução mensal da gasolina por estado:")
print(evolucao_gasolina.head())

print("\nComparação gasolina comum e aditivada:")
print(comparativo.head())

print("\nVariação por região:")
print(variacao_regiao.head())

print("\nPostos com maiores preços (Gasolina comum):")
print(maiores_precos)

print("\nPostos com menores preços (Gasolina comum):")
print(menores_precos)

print("\nEstado com maior variação percentual na gasolina (2020-2024):")
print(maior_var)

print("\nEstados onde etanol é competitivo (menos de 70% do preço da gasolina):")
print(estados_competitivos)

print("\nMédia de preços por distribuidora (Gasolina comum):")
print(media_dist.head())

NameError: name 'df' is not defined